In [15]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
import pandas as pd
from tensorflow.keras.models import load_model
from mltu.configs import BaseModelConfigs
from tqdm import tqdm
from mltu.utils.text_utils import ctc_decoder, get_cer
from mltu.inferenceModel import OnnxInferenceModel

In [17]:
# Custom OCR model
class ImageToWordModel(OnnxInferenceModel):
    def __init__(self, char_list, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.char_list = char_list

    def predict(self, image):
        image = cv2.resize(image, self.input_shapes[0][1:3][::-1])
        image_pred = np.expand_dims(image, axis=0).astype(np.float32)
        preds = self.model.run(self.output_names, {self.input_names[0]: image_pred})[0]
        text = ctc_decoder(preds, self.char_list)[0]
        return text

In [18]:
# Load pre-trained CNN model and make predictions
def tester(model, img):
    img_resized = cv2.resize(img, (128, 128))  # Resize to the input size of the CNN
    img_resized = img_resized.astype('float32') / 255.0  # Normalize the image
    img_resized = np.expand_dims(img_resized, axis=0)  # Add batch dimension (1, 128, 128, 3)

    prediction = model.predict(img_resized)
    predicted_class = np.argmax(prediction, axis=1)[0]  # Get class index
    class_labels = ['Handwritten', 'Printed']
    print(f"Predicted class: {class_labels[predicted_class]}")
    return class_labels[predicted_class]

In [19]:
# Custom OCR function using ImageToWordModel
def perform_ocr(img, ocr_model):
    prediction_text = ocr_model.predict(img)
    print(f"Recognized Handwritten Text: {prediction_text}")
    return prediction_text

In [20]:
# Function to clear the output directory
def clear_output_directory(output_dir):
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)

In [21]:
# Classification of Prescription Image Sections
def Classify(filename):
    output_dir = 'output'
    clear_output_directory(output_dir)

    img = cv2.imread(filename)
    hgt, wdt = img.shape[:2]
    hBw = hgt / float(wdt)
    dim = (576, int(576 * hBw))
    
    img = cv2.resize(img, dim)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    linek = np.zeros((11, 11), dtype=np.uint8)
    linek[5, ...] = 1
    x = cv2.morphologyEx(gray, cv2.MORPH_OPEN, linek, iterations=1)
    gray -= x

    ret2, gray = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((5, 5), np.uint8)
    gray = cv2.dilate(gray, kernel, iterations=1)

    contours2, _ = cv2.findContours(gray, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    cnn_model = load_model("cnn_model_images.h5")  # Load the trained CNN model

    configs = BaseModelConfigs.load("Models/03_handwriting_recognition/202301111911/configs.yaml")
    ocr_model = ImageToWordModel(model_path=configs.model_path, char_list=configs.vocab)

    file_counter = 1
    for cnt in contours2:
        start_x, start_y, width, height = cv2.boundingRect(cnt)
        mymat = img[start_y:start_y + height, start_x:start_x + width]
        dect = tester(cnn_model, mymat)

        if dect == 'Handwritten':
            ocr_result = perform_ocr(mymat, ocr_model)
            
            sanitized_text = "".join([c if c.isalnum() else "_" for c in ocr_result]) if ocr_result else "unrecognized"
            region_filename = f"{output_dir}/{sanitized_text[:30]}_{file_counter}.png"
            
            cv2.imwrite(region_filename, mymat)
            print(f"Saved {region_filename}")
            file_counter += 1
        
        color = (0, 255, 0) if dect == 'Handwritten' else (255, 0, 0)
        cv2.rectangle(img, (start_x, start_y), (start_x + width, start_y + height), color, 2)
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()
    cv2.imwrite("output/classified_image.png", img)

In [22]:
# Execution - Classify a New Image
if __name__ == "__main__":
    Classify("Templates/7.jpg")

E0000 00:00:1731603782.215527     350 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
E0000 00:00:1731603782.225101     350 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2024-11-14 22:33:02.228557: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at xla_ops.cc:577 : FAILED_PRECONDITION: DNN library initialization failed. Look at the er

FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/harsh/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/harsh/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/harsh/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/harsh/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/harsh/.local/lib/python3.10/site-packages/ipykernel/kernelbase.py", line